# Imports

In [1]:
import gspread_pandas

# Load data

In [2]:
spread = gspread_pandas.Spread("search_retired1.search_export.gsheet")
df = spread.sheet_to_df(index=None)
print(df.shape)
df.head()

(14, 19)


,profileUrl,connectionDegree,error,currentJob,job,location,url,query,category,timestamp,fullName,firstName,lastName,profileImageUrl,summary,name,additionalInfo,pastJob,sharedConnections
0,,Out of Network,Profile out of your network.,Retired (Supervisory Environmental Engineer) a...,--,"Bel Air, MD",,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.384Z,,,,,,,,,
1,https://www.linkedin.com/in/yale-lewis-mba-049...,3rd,,,Retired in 2020,"Winchester, VA",https://www.linkedin.com/in/yale-lewis-mba-049...,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.384Z,"Yale Lewis, MBA",Yale,"Lewis, MBA",https://media.licdn.com/dms/image/C4D03AQGGXPi...,...Administration (MBA) Specialties: Financi...,"Yale Lewis, MBA",,,
2,https://www.linkedin.com/in/richard-saunders-9...,3rd,,,Retired from Federal Service.,"Silver Spring, MD",https://www.linkedin.com/in/richard-saunders-9...,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.384Z,Richard Saunders,Richard,Saunders,https://media.licdn.com/dms/image/D4E03AQHxFXh...,,Richard Saunders,,,
3,,Out of Network,Profile out of your network.,,"Retired, trying to figure out what's next","New Alexandria, VA",,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.384Z,,,,https://media.licdn.com/dms/image/C4E03AQEhKLx...,,,,,
4,https://www.linkedin.com/in/robertfarinella,3rd,,Retired at And Enjoying Life,US Army and Civilian Hospitality/Gaming/Horse ...,"Culpeper, VA",https://www.linkedin.com/in/robertfarinella,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.385Z,"Robert Farinella, MBA",Robert,"Farinella, MBA",https://media.licdn.com/dms/image/C4E03AQHPttO...,,"Robert Farinella, MBA",,,


# Filter data

In [3]:
# Drop errors.
prev_len = len(df)
df = df[df["error"] == ""].reset_index(drop=True)
df = df[[col for col in df.columns if col != "error"]]
print(
    f"Dropped {prev_len - len(df)} rows ({round((prev_len - len(df))*100/prev_len, 2)}%)"
)

Dropped 4 rows (28.57%)


In [4]:
# Filter by location.
prev_len = len(df)
df = df[df["location"].apply(lambda x: "MD" in x)].reset_index(drop=True)
print(
    f"Dropped {prev_len - len(df)} rows ({round((prev_len - len(df))*100/prev_len, 2)}%)"
)

Dropped 4 rows (40.0%)


In [5]:
print(df.shape)
df

(6, 18)


,profileUrl,connectionDegree,currentJob,job,location,url,query,category,timestamp,fullName,firstName,lastName,profileImageUrl,summary,name,additionalInfo,pastJob,sharedConnections
0,https://www.linkedin.com/in/richard-saunders-9...,3rd,,Retired from Federal Service.,"Silver Spring, MD",https://www.linkedin.com/in/richard-saunders-9...,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.384Z,Richard Saunders,Richard,Saunders,https://media.licdn.com/dms/image/D4E03AQHxFXh...,,Richard Saunders,,,
1,https://www.linkedin.com/in/scottalawrence,3rd,,"Senior Manager, Software Engineering at Capita...","Silver Spring, MD",https://www.linkedin.com/in/scottalawrence,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.385Z,Scott L.,Scott,L.,https://media.licdn.com/dms/image/C4E03AQHI9E9...,,Scott L.,Volunteer Experience: Instructor at Adventist ...,,
2,https://www.linkedin.com/in/perrycoleman1,3rd,,"IT Specialist, MBA, MSCIS, PMP, CISSP, DML","Gwynn Oak, MD",https://www.linkedin.com/in/perrycoleman1,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.385Z,Perry M Coleman,Perry,M Coleman,https://media.licdn.com/dms/image/C4E03AQEJYFZ...,,Perry M Coleman,,Unmanned Aircraft System Operator at US Army -...,
3,https://www.linkedin.com/in/kathleenmbovard,3rd,,"Skilled career coach, facilitator and mentor c...","Baltimore, MD",https://www.linkedin.com/in/kathleenmbovard,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:51.385Z,Kathleen Bovard,Kathleen,Bovard,https://media.licdn.com/dms/image/C4E03AQHfUCO...,,Kathleen Bovard,,,
4,https://www.linkedin.com/in/christopher-a-tayl...,3rd,,SCORE Mentor & Retired Head of Healthcare & Fi...,"Baltimore, MD",https://www.linkedin.com/in/christopher-a-tayl...,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:58.059Z,Christopher A. Taylor,Christopher,A. Taylor,https://media.licdn.com/dms/image/C4D03AQEXytm...,,Christopher A. Taylor,,,
5,https://www.linkedin.com/in/dr-donald-jg-chiar...,3rd,,CEO Chiarella Consulting,"Hanover, MD",https://www.linkedin.com/in/dr-donald-jg-chiar...,https://www.linkedin.com/search/results/people...,People,2023-07-13T15:51:58.059Z,"Dr. Donald JG Chiarella, CPEM, CHPP, CISM, CDP...",Dr.,"Donald JG Chiarella, CPEM, CHPP, CISM, CDP, CS...",https://media.licdn.com/dms/image/C5603AQGZXgW...,"..., PEM, Scrum, and ITIL for clients. Retired...","Dr. Donald JG Chiarella, CPEM, CHPP, CISM, CDP...",,,


# Save filtered data to gsheet

In [6]:
spread2 = gspread_pandas.Spread(
    "search_retired1.search_export.filtered.gsheet",
    sheet="Sheet1",
    create_sheet=True,
)
spread2.df_to_sheet(df, index=False)